In [ ]:
import json
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import warnings

In [ ]:
train = json.load(open('/kaggle/input/pii-detection-removal-from-educational-data/train.json'))


In [ ]:
labels = []
tokens = []
for i in train:
    labels.extend([j for j in i['labels'] if j!='O'])
    tokens.extend([k for j, k in zip(i['labels'],i['tokens']) if j!='O'])


<div style="color:white;
       display:fill;
       border-radius:5px;
       background-color:#5642C5;
       font-size:110%;
       font-family:Verdana;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:white;"> Eğer etiket 'O' değilse, yalnızca bu etiketi labels listesine ekliyor. labels listesi 'O' olmayan tüm etiketleri içerir. tokens listesi, bu 'O' olmayan etiketlere ait olan tokenları içerir. 

In [ ]:
data = pd.DataFrame({'labels':labels,'tokens':tokens})
data.head()

# Label Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.DataFrame({'labels': labels, 'tokens': tokens})

label_counts = data['labels'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette='viridis')
plt.title('distribution')
plt.xlabel('labels')
plt.ylabel('counts')
plt.xticks(rotation=45)
plt.show()


# Distribution of B and I Labels

In [ ]:
data['BIO'] = data['labels'].apply(lambda x: x[0])

In [ ]:
import matplotlib.pyplot as plt

# Veri çerçevesinin başlangıç kısmını oluşturma
data = pd.DataFrame({'labels': labels, 'tokens': tokens})

# B ve I etiketlerinin sayısını hesapla
b_count = data[data['labels'].str.startswith('B-')]['labels'].count()
i_count = data[data['labels'].str.startswith('I-')]['labels'].count()

# Pie chart oluştur
labels = ['B', 'I']
sizes = [b_count, i_count]
colors = ['skyblue', 'lightgreen']
explode = (0.1, 0)  # İlk dilim (B) birazcık dışa doğru taşsın

plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('B ve I Etiketlerinin Dağılımı')
plt.axis('equal')  # Dairesel şeklin korunması için
plt.show()


In [ ]:
data['BIO'] = data['labels'].apply(lambda x: x[0])


In [ ]:
BIO_list = data['BIO']
lengths = []
ids = []
for i, j in enumerate(BIO_list):
    if i == 0:
        ids.append(0)
        lengths.append(1)
    else:
        if (j == 'I'):
            ids.append(ids[i-1])
            lengths.append(lengths[i-1]+1)
        else:
            ids.append(ids[i-1]+1)
            lengths.append(1)
data['lengths'] = lengths
data['ids'] = ids
data

In [ ]:
length_data = data.groupby('ids').agg({"lengths":"max", "tokens":lambda x:' '.join(x), "labels":lambda x:x.tolist()[0][2:]})
length_data.head()

In [ ]:
length_data['labels'].unique()

In [ ]:
length_data = data.groupby('ids').agg({"lengths":"max", "tokens":lambda x:' '.join(x), "labels":lambda x:x.tolist()[0][2:]})
length_data.head()

# Token Lenghts

In [ ]:
max_length = data['lengths'].max()
min_length = data['lengths'].min()

print("Longest token length:", max_length)
print("Shortest token length:", min_length)


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['lengths'], bins=20, kde=False)
plt.title('Distribution of Token Lengths')
plt.xlabel('Token Lengths')
plt.ylabel('Frequency')
plt.show()


In [ ]:
token_length_means = data.groupby('labels')['lengths'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='labels', y='lengths', data=token_length_means)
plt.title('Average Token Lengths by Label Types')
plt.xlabel('Labels')
plt.ylabel('Average Token Lengths')
plt.xticks(rotation=45)
plt.show()


# Most Frequently Used Tokens

In [ ]:
token_labels = data.groupby('tokens')['labels'].agg(lambda x: x.mode()[0])

label_counts = token_labels.value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette='viridis')
plt.title('Labels of the Most Frequently Used Tokens')
plt.xlabel('Labels')
plt.ylabel('Token Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:
token_counts = data['tokens'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=token_counts.index[:20], y=token_counts.values[:20], palette='viridis')
plt.title('Most Frequently Used Tokens')
plt.xlabel('Tokens')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


# Average Token Counts

In [ ]:
def calculate_token_count(document_tokens):
    return len(document_tokens)

document_token_counts = data['tokens'].apply(lambda x: calculate_token_count(x))

average_token_count = document_token_counts.mean()

print("Average Number of Tokens in Documents:", average_token_count)


B ve I etiketlerini birleştirmeden önceki veri setine göre ortalama token sayısı

In [ ]:
def calculate_token_count(document_tokens):
    return len(document_tokens)

document_token_counts = length_data['tokens'].apply(lambda x: calculate_token_count(x))

average_token_count = document_token_counts.mean()

print("Average Number of Tokens in Documents:", average_token_count)


B ve I etiketlerini birleştirdikten sonraki veri setine göre ortalama token sayısı

# Length_data 

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=length_data, x='lengths', bins=30, kde=False, color='skyblue')
plt.title('Distribution of Token Lengths')
plt.xlabel('Token Lengths')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=length_data, x='labels', palette='viridis')
plt.title('Labels Distribution')
plt.xlabel('Labels')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


# Longest Tokens and Character Counts

In [ ]:
# top 20 longest names
length_data[length_data['labels']=='NAME_STUDENT'].sort_values('lengths', ascending = False).head(20)

In [ ]:
name_student_tokens = length_data[length_data['labels'] == 'NAME_STUDENT']

token_lengths = name_student_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=range(min(token_lengths), max(token_lengths) + 1), color='skyblue', edgecolor='black')
plt.title('Distribution of NAME_STUDENT Tokens by Character Count')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.xticks(range(min(token_lengths), max(token_lengths) + 1))
plt.grid(True)
plt.show()


In [ ]:
# top 5 longest urls
length_data[length_data['labels']=='URL_PERSONAL'].sort_values('lengths', ascending = False).head(5)

In [ ]:
url_personal_tokens = length_data[length_data['labels'] == 'URL_PERSONAL']

token_lengths = url_personal_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Token Lengths for URL_PERSONAL Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# top 5 longest emails
length_data[length_data['labels']=='EMAIL'].sort_values('lengths', ascending = False).head(5)

In [ ]:
email_tokens = length_data[length_data['labels'] == 'EMAIL']

token_lengths = email_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Character Count for EMAIL Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# top 5 longest ID_NUM
length_data[length_data['labels']=='ID_NUM'].sort_values('lengths', ascending = False).head(5)

In [ ]:
id_num_tokens = length_data[length_data['labels'] == 'ID_NUM']

token_lengths = id_num_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Character Count for ID_NUM Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# top 5 longest usernames
length_data[length_data['labels']=='USERNAME'].sort_values('lengths', ascending = False).head(5)

In [ ]:
username_tokens = length_data[length_data['labels'] == 'USERNAME']

token_lengths = username_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Character Count for USERNAME Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# top 10 longest phone numbers
length_data[length_data['labels']=='PHONE_NUM'].sort_values('lengths', ascending = False).head(10)

In [ ]:
phone_num_tokens = length_data[length_data['labels'] == 'PHONE_NUM']

token_lengths = phone_num_tokens['tokens'].apply(len)

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Character Count for PHONE_NUM Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# top 10 longest address
length_data[length_data['labels']=='STREET_ADDRESS'].sort_values('lengths', ascending = False).head(10)

In [ ]:
street_address_tokens = length_data[length_data['labels'] == 'STREET_ADDRESS']

character_lengths = street_address_tokens['tokens'].apply(len)

character_lengths_counts = character_lengths.value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(character_lengths_counts.index, character_lengths_counts.values, color='skyblue')
plt.title('Distribution of Character Count for STREET_ADDRESS Tokens')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.grid(axis='y')
plt.xticks(character_lengths_counts.index)
plt.show()


**Thank you!**